<div style="width: 38.5%;">
    <p><strong>City College of San Francisco</strong><p>
    <hr>
    <p>MATH 108 - Foundations of Data Science</p>
</div>

# Lecture 22: Examples

Associated Textbook Sections: [12.3](https://inferentialthinking.com/chapters/12/3/Deflategate.html)

## Outline

* [Benford's Law](#Benford's-Law)
* [Reaction Time](#Reaction-Time)
* [Zodiac Signs](#Zodiac-Signs)

## Set Up the Notebook

In [ ]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

---

## Benford's Law

> [Benford's law](https://en.wikipedia.org/wiki/Benford%27s_law), also known as the Newcomb-Benford law, the law of anomalous numbers, or the first-digit law, is an observation that in many real-life sets of numerical data, the leading digit is likely to be small. Benford's law tends to apply most accurately to data that span several orders of magnitude. As a rule of thumb, the more orders of magnitude that the data evenly covers, the more accurately Benford's law applies.

Observe the distribution of the first digits of numbers according to Benford's model.

In [ ]:
digits = np.arange(1, 10)
benford_model = np.log10(1 + 1/digits)
benford = Table().with_columns(
    'First digit', digits,
    'Benford model prob', benford_model)
benford

In [ ]:
benford.barh('First digit')

Use bracket notation to get get the first character of a string. 

_The use of bracket notation here is just for an example. Bracket notation is a common way in Python to obtain information within a collection. This is similar to how we use `.item(0)` with arrays._

In [ ]:
a_string = 'data science'
a_string[0]

Try the same thing with an integer.

In [ ]:
# Uncomment this to see a TypeError
an_integer = 1234
#an_integer[0]

Explore the `first_digit` function.

In [ ]:
def first_digit(num):
    """Returns the first digit of the interger num."""
    return int(str(num)[0])

In [ ]:
first_digit(32)

Load the `counties.csv` data. This data contains county population sizes from the 2010 Census.

In [ ]:
counties = Table.read_table('./data/counties.csv')
counties = counties.where('SUMLEV', 50).select(5,6,9)\
                                       .relabeled(0,'State')\
                                       .relabeled(1,'County')\
                                       .relabeled(2,'Population')

counties

In [ ]:
counties.where('County', 'San Francisco County')

Apply `first_digit` to add a column to the `countries` table that shows the first digit of the population sizes.

In [ ]:
first_digits = counties.apply(first_digit, 'Population')
counties = counties.with_column('First digit', first_digits)
counties.show(3)

Visually compare the distribution of first digits from the countries data and Benford's proportions.

In [ ]:
num_counties = counties.num_rows
by_digit = counties.group('First digit')
proportions = by_digit.column('count') / num_counties
by_digit = by_digit.with_columns(
    'Proportion', proportions,
    'Benford proportion', benford_model
)
by_digit.drop('count').barh('First digit')

Test whether or not the distribution of proportions in `countries` is consistent with Benford's model.

Null hypothesis: ...

Alternative hypothesis: ...

Test statistic: ...

Fill in the blank with "Bigger" or "Smaller":

... values of the test statistic favor the alternative

In [ ]:
# Calculate the observed statistic
...

In [ ]:
# Demonstrate how to create simulated random samples
...

In [ ]:
# Demonstrate how to calculate the test statistic for a simulated random sample
...

In [ ]:
# Create a function that creats a simulated random sample and calculates the test statistic.
def simulate_county_first_digits():
    ...
    return ...

In [ ]:
# Create an array of 10,000 simulated statistics.

In [ ]:
# Visualize the emperical distribution for the simulated statistics
Table().with_column(...)
plots.scatter(observed_tvd, 0, color='red', s=60, zorder=3)
plots.show()

In [ ]:
# Calculate the p-value

Are the data consistent with the null hypothesis?

...

---

## Survey Observations

* _The following examples use a real data set (Class Survey) along with the hypothesis testing procedure to come to conclusions._
* _Keep in mind that data is not form a random sample._
* _The conclusions we reach should no te taken seriously._

### Reaction Time

Load our welcome survey data from `welcome_survey_sp23.csv`.

In [ ]:
survey = Table.read_table('./data/welcome_survey_sp23.csv')
survey.show(3)

Create a reduced table by selecting the columns:
* `Student Status'`
* `'Reaction Time (ms)'`

In [ ]:
status_reaction = (survey.select('Student Status', 'Reaction Time (ms)')
                   .where('Reaction Time (ms)', are.above_or_equal_to(0))
                   .where('Student Status', are.not_equal_to('nan')))
status_reaction

In [ ]:
status_reaction.group('Student Status')

In [ ]:
status_reaction.hist('Reaction Time (ms)', group='Student Status')

Determine the average reaction time for the two different groups.

In [ ]:
status_reaction.group('Student Status', np.average)

Test whether or not there is a significant difference between the average reaction time for those that have a bachelor degree or do not.

Null hypothesis: **The average reaction time for full-time and part-time students is the same.**

Alternative hypothesis: **The average reaction time for full-time and part-time students is not the same.**

Test statistic: **Absolute difference in the average reaction time for the two groups.**

Fill in the blank with "Bigger" or "Smaller":

**Bigger** values of the test statistic favor the alternative

In [ ]:
def compute_test_statistic(tbl):
    grouped = tbl.group('Student Status', np.average)
    avgs = grouped.column('Reaction Time (ms) average')
    return abs(avgs.item(1) - avgs.item(0))

In [ ]:
obs_test_stat = compute_test_statistic(status_reaction)
obs_test_stat

In [ ]:
random_labels = status_reaction.sample(with_replacement=False).column('Student Status')
random_labels

In [ ]:
def simulate_under_null():
    random_labels = status_reaction.sample(with_replacement=False).column('Student Status')
    relabeled_tbl = status_reaction.with_column('Student Status', random_labels)
    return compute_test_statistic(relabeled_tbl)


In [ ]:
simulated_diffs = make_array()

for __ in np.arange(1000):
    null_stat = simulate_under_null()
    simulated_diffs = np.append(simulated_diffs, null_stat)

In [ ]:
Table().with_column('Simulated difference', simulated_diffs).hist(0)
plots.scatter(obs_test_stat, 0, color='red', s=60, zorder=3)
plots.show()

In [ ]:
np.mean(simulated_diffs <= obs_test_stat)

Are the results statistically significant?

* **With a p-value larger than 5%, the results are not statistically significant.**
* Notice that the choice of statistic doesn't allow us to see the difference in the shape of the distributions.

---

## Zodiac Signs

Load the distribution of Zodiac signs in the United States (as found in 2018).

_Source: https://www.statisticbrain.com/zodiac-sign-statistics_

In [ ]:
zodiac_distribution = Table.read_table('./data/zodiac_distribution.csv')
zodiac_distribution.show()

In [ ]:
zodiac_distribution.barh('Zodiac Sign')

Get the birthdays from the survey data, convert the birthdays to Zodiac signs, and determien the distribution of Zodiac signs in MATH 108.

In [ ]:
birthdays = survey.select('Birthday').where('Birthday', are.not_equal_to('nan'))
birthdays

In [ ]:
def get_zodiac_sign(birthday):
    """
    Given a birthday in the format "MM/DD", 
    returns a string representing the corresponding zodiac sign.
    """
    month, day = map(int, birthday.split("/")) # map is not covered in this class
    if month == 12:
        # a more compact way of writing an if statement
        astro_sign = 'Sagittarius' if (day < 22) else 'Capricorn' 
    elif month == 1:
        astro_sign = 'Capricorn' if (day < 20) else 'Aquarius'
    elif month == 2:
        astro_sign = 'Aquarius' if (day < 19) else 'Pisces'
    elif month == 3:
        astro_sign = 'Pisces' if (day < 21) else 'Aries'
    elif month == 4:
        astro_sign = 'Aries' if (day < 20) else 'Taurus'
    elif month == 5:
        astro_sign = 'Taurus' if (day < 21) else 'Gemini'
    elif month == 6:
        astro_sign = 'Gemini' if (day < 21) else 'Cancer'
    elif month == 7:
        astro_sign = 'Cancer' if (day < 23) else 'Leo'
    elif month == 8:
        astro_sign = 'Leo' if (day < 23) else 'Virgo'
    elif month == 9:
        astro_sign = 'Virgo' if (day < 23) else 'Libra'
    elif month == 10:
        astro_sign = 'Libra' if (day < 23) else 'Scorpio'
    elif month == 11:
        astro_sign = 'Scorpio' if (day < 22) else 'Sagittarius'
    else:
        astro_sign = 'Invalid Date'
    return astro_sign

In [ ]:
def prop_function(count):
    """Return the count provided as a proportion of 40 (# of respondents)."""
    total_number = 40
    return count/40

birthdays = birthdays.with_column(
    'Zodiac Sign',
    birthdays.apply(get_zodiac_sign, 'Birthday')
)

birthdays_by_sign = birthdays.group('Zodiac Sign')
birthdays_by_sign = birthdays_by_sign.with_column(
    'Proportion in MATH 108',
    birthdays_by_sign.apply(prop_function, 'count')
).drop('count').sort('Zodiac Sign')

zodiac = birthdays_by_sign.join('Zodiac Sign', zodiac_distribution)
zodiac.show()

In [ ]:
zodiac.barh('Zodiac Sign')

Run a test to see if there is a significant difference between the distribution of Zodiac signs in the class and the nation.

In [ ]:
observed_tvd_zodiac = np.sum(np.abs(zodiac.column('Proportion in MATH 108')
                              - zodiac.column('Proportion of US Population')))/2
observed_tvd_zodiac

In [ ]:
def simulate_zodiac_proportions():
    simulated_proportions = sample_proportions(40, zodiac.column('Proportion of US Population'))
    tvd = np.sum(np.abs(simulated_proportions - zodiac.column('Proportion of US Population'))) / 2
    return tvd

simulated_tvds_zodiac = make_array()

for __ in np.arange(10000):
    simulated_tvds_zodiac = np.append(simulated_tvds_zodiac, simulate_zodiac_proportions())
    
Table().with_column('Simulated TVD', simulated_tvds_zodiac).hist(0)
plots.scatter(observed_tvd_zodiac, 0, color='red', s=60, zorder=3)
plots.show()

np.count_nonzero(simulated_tvds_zodiac >= observed_tvd_zodiac) / 10000

---

## Test Reflection

* The p-value for the last test wass almost 70%. 
* This means that our test results are NOT statistically significant. 🫣
* The course Zodiac distribution looks wildly different form populations distribution.
* How can this be? 
    * Sample size has a pretty big impact. 
    * Try changing the sample size in `sample_proportions` to 400, instead of 40.

### Hypothesis Test Concerns

The outcome of a hypothesis test can be affected by:
* The hypotheses you investigate: 
    * How do you define your null distribution?
* The test statistic you choose: 
    * How do you measure a difference between samples?
* The empirical distribution of the statistic under the null:
    * How many times do you simulate under the null distribution?
* The data you collected:
    * Did you happen to collect a sample that is similar to the population?
* The truth:
    * If the alternative hypothesis is true, how extreme is the difference?

### Hypothesis Test Effects

* Number of simulations: 
    * large as possible: empirical distribution → true distribution
    * No new data needs to be collected (yay!)
* Number of observations: 
    * A larger sample will lead you to reject the null more reliably if the alternative is in fact true.
* Difference from the null: 
    * If truth is similar to the null hypothesis, then even a large sample may not provide enough evidence to reject the null.


<footer>
    <p>Adopted from UC Berkeley DATA 8 course materials.</p>
    <p>This content is offered under a <a href="https://creativecommons.org/licenses/by-nc-sa/4.0/">CC Attribution Non-Commercial Share Alike</a> license.</p>
</footer>